In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.formula.api as smf

In [68]:
lr_data = pd.read_csv('CDC-2019-2023-DATA_nums.csv', low_memory=False)

In [69]:
lr_data.columns

Index(['Unnamed: 0', 'BIRTHSEX', 'MENTHLTH', 'POORHLTH', 'ADDEPEV3', 'DECIDE',
       'DIFFALON', 'IYEAR', 'ACEDEPRS', 'ACEDRINK', 'ACEDRUGS', 'ACEPRISN',
       'ACEDIVRC', 'ACEPUNCH', 'ACEHURT1', 'ACESWEAR', 'ACETOUCH', 'ACETTHEM',
       'ACEHVSEX', 'EMPLOY1', 'AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH'],
      dtype='object')

In [70]:
lr_data = lr_data.drop(['Unnamed: 0'], axis=1)

In [127]:
lr_data['ACESWEAR'].value_counts()

ACESWEAR
Never             16599
More than once    11363
Once               2039
Name: count, dtype: int64

In [117]:
ace_fig = px.box(lr_data,x='ACESWEAR',y="MENTHLTH", color='ACESWEAR')

ace_fig.show()

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression


In [75]:
lr_data = lr_data.dropna(subset=['MENTHLTH', 'POORHLTH', 'IYEAR', 'ACESWEAR',
            'EMPLOY1', 'AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH'])

In [76]:
print(lr_data.shape)

(30001, 23)


In [84]:
import statsmodels.formula.api as smf

In [85]:
model = smf.ols(formula="MENTHLTH~POORHLTH + IYEAR + ACESWEAR + EMPLOY1 + AVEDRNK3 + EXEROFT1 + STRENGTH + PHYSHLTH", data=lr_data).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               MENTHLTH   R-squared:                       0.156
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     368.9
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        12:35:06   Log-Likelihood:            -1.0550e+05
No. Observations:               30001   AIC:                         2.110e+05
Df Residuals:                   29985   BIC:                         2.112e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

In [77]:
y = lr_data['MENTHLTH']
X = lr_data[['POORHLTH', 'IYEAR', 'ACESWEAR',
            'EMPLOY1', 'AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH']]
        

In [78]:
nums = ['AVEDRNK3', 'EXEROFT1', 'STRENGTH', 'PHYSHLTH', 'POORHLTH']
cats = ['IYEAR', 'ACESWEAR', 'EMPLOY1']

In [79]:
preprocess = ColumnTransformer(transformers=[('encoder',OneHotEncoder(drop='first'),cats),
                                             ('numeric','passthrough',nums)])

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42,stratify=y)

In [89]:
ols_model = Pipeline([("preprocess", preprocess),
        ("linreg", LinearRegression())
])


poly3_model = Pipeline([
    ("preprocess", preprocess),
    ("poly", PolynomialFeatures(degree=3, include_bias=False)),
    ("scaler", StandardScaler()),
    ("linreg", LinearRegression())
])


spline_model = Pipeline([
    ("preprocess", preprocess),
    ("spline", SplineTransformer(degree=3, n_knots=8, include_bias=False)),
    ("linreg", LinearRegression())
])

models = {
    "OLS (linear)": ols_model,
    "Polynomial (cubic)": poly3_model,
    "Cubic spline (n_knots=8)": spline_model
}


In [90]:
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

rows = []
for name, model in models.items():
    model.fit(X_train, y_train)

    yhat_tr = model.predict(X_train)
    yhat_te = model.predict(X_test)

    rows.append({
        "Model": name,
        "Train RMSE": rmse(y_train, yhat_tr),
        "Test RMSE": rmse(y_test, yhat_te),
        "Train R2": r2_score(y_train, yhat_tr),
        "Test R2": r2_score(y_test, yhat_te),
    })

results = pd.DataFrame(rows).sort_values("Test RMSE")
print(results.round(4))

                      Model  Train RMSE  Test RMSE  Train R2  Test R2
2  Cubic spline (n_knots=8)      7.9240     7.9355    0.2013   0.1984
0              OLS (linear)      8.1503     8.1340    0.1550   0.1578
1        Polynomial (cubic)      7.7408     9.6287    0.2378  -0.1801


In [97]:
aces_categories = sorted(lr_data['ACESWEAR'].unique())

In [ ]:
# Start with a row of means/mode
base = {}

for col in nums:
    base[col] = lr_data[col].mean()

for col in cats:
    base[col] = lr_data[col].mode()[0]

In [98]:
plot_df = pd.DataFrame([base.copy() for _ in aces_categories])
plot_df['ACESWEAR'] = aces_categories

In [99]:
y_pred_spline = spline_model.predict(plot_df)

In [129]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=aces_categories,
    y=y_pred_spline,
))

fig.update_layout(
    title="Cubic Spline Prediction of Bad Mental Health Days by Frequency of Adverse Childhood Experience",
    xaxis_title="ACESWEAR (Adverse Childhood Experience Score)",
    yaxis_title="Predicted MENTHLTH",
)

fig.show()